In [10]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset/Amazon_Product_Review_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_body'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên
data['star_rating'] = data['star_rating'].astype(int)

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())

# 5. Tách dữ liệu thành đặc trưng (X) và nhãn (y)
X = data['review']
y = data['star_rating']

# 6. Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Trích xuất đặc trưng từ văn bản bằng TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 8. Xây dựng và huấn luyện mô hình SVM (phân loại đa lớp: 1,2,3,4,5)
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_tfidf, y_train)




# #không có GridSearchCV + Smote + Hyperparameter, kernel: linear -> Accurancy: 0.71
# #không có GridSearchCV + Smote + Hyperparameter, kernel: rbf -> Accurancy: 0.70
# #Có GridSearchCV + Smote + Hyperparameter, kernel: linear class_weight='balanced' -> Accurancy: 0.6-0.64
# #Có GridSearchCV + Smote + Hyperparameter, kernel: rbf class_weight='balanced' -> Accurancy: 0.64




Các cột trong dataset: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Các star_rating độc nhất: [5 1 3 4 2]


SVC(kernel='linear', random_state=42)

In [11]:
# 9. Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test_tfidf)

print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred), "\n")
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))

Độ chính xác (Accuracy): 0.7045380875202594 

Ma trận nhầm lẫn:
[[ 175   18   43   26   82]
 [  60   26   51   39   61]
 [  56   20  110  119  157]
 [  24    8   80  279  768]
 [  35    2   28  146 3757]] 

Báo cáo phân loại:
              precision    recall  f1-score   support

           1       0.50      0.51      0.50       344
           2       0.35      0.11      0.17       237
           3       0.35      0.24      0.28       462
           4       0.46      0.24      0.32      1159
           5       0.78      0.95      0.85      3968

    accuracy                           0.70      6170
   macro avg       0.49      0.41      0.43      6170
weighted avg       0.65      0.70      0.66      6170



In [19]:

# 9. Kiểm thử với các câu thực tế
test_sentences = [
"Sản phẩm rất tốt, tôi hài lòng!",
"Không đáng tiền, rất thất vọng.",
"Amazing quality, totally worth it!",
"Terrible experience, never buying again.",
"Sản phẩm ổn, nhưng giao hàng chậm.",
"I am not happy with this product at all.",
"Tôi không thích sản phẩm này",
"Sản phẩm rất tệ",
"Sản phẩm rất tốt",
"Giao hàng nhanh, đóng gói cẩn thận.",
"Dịch vụ chăm sóc khách hàng quá tệ.",
"Tôi mong đợi nhiều hơn từ sản phẩm này.",
"Tuyệt vời! Tôi sẽ mua lại lần nữa.",
"Quá thất vọng, hàng không như mô tả.",
"It's okay, but a bit overpriced.",
"Sản phẩm ổn, nhưng giao hàng chậm.",
"It's okay, but a bit overpriced.", 
"Sản phẩm rất tệ, không như quảng cáo!" ,
"Worst purchase ever. Completely useless.", 
"Hàng lỗi, không sử dụng được, quá thất vọng.",
"Hàng chất lượng tạm ổn, nhưng đóng gói chưa cẩn thận."
]

X_test_sentences = tfidf.transform(test_sentences)
y_test_pred = best_svm.predict(X_test_sentences)

# Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")

Review: Sản phẩm rất tốt, tôi hài lòng! -> Dự đoán: 1 sao
Review: Không đáng tiền, rất thất vọng. -> Dự đoán: 5 sao
Review: Amazing quality, totally worth it! -> Dự đoán: 4 sao
Review: Terrible experience, never buying again. -> Dự đoán: 5 sao
Review: Sản phẩm ổn, nhưng giao hàng chậm. -> Dự đoán: 4 sao
Review: I am not happy with this product at all. -> Dự đoán: 4 sao
Review: Tôi không thích sản phẩm này -> Dự đoán: 5 sao
Review: Sản phẩm rất tệ -> Dự đoán: 2 sao
Review: Sản phẩm rất tốt -> Dự đoán: 5 sao
Review: Giao hàng nhanh, đóng gói cẩn thận. -> Dự đoán: 5 sao
Review: Dịch vụ chăm sóc khách hàng quá tệ. -> Dự đoán: 5 sao
Review: Tôi mong đợi nhiều hơn từ sản phẩm này. -> Dự đoán: 5 sao
Review: Tuyệt vời! Tôi sẽ mua lại lần nữa. -> Dự đoán: 4 sao
Review: Quá thất vọng, hàng không như mô tả. -> Dự đoán: 5 sao
Review: It's okay, but a bit overpriced. -> Dự đoán: 5 sao
Review: Sản phẩm ổn, nhưng giao hàng chậm. -> Dự đoán: 4 sao
Review: It's okay, but a bit overpriced. -> Dự đoán: 5